In [ ]:
!pip install jupyter-dash plotly

In [ ]:
import pandas as pd
import plotly.express as px
import funcs

df = pd.read_csv("./df.csv")
df_pl = pd.read_csv("./df_pl.csv")

df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
df_pl = df_pl.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [ ]:
genre_list_pl = list(df_pl['genre_1'].unique()) + list(df_pl['genre_2'].unique()) + list(df_pl['genre_3'].unique()) + list(df_pl['genre_4'].unique()) + list(df_pl['genre_5'].unique())
genre_set_pl = set(genre_list_pl)

genre_list = list(df['genre_1'].unique()) + list(df['genre_2'].unique()) + list(df['genre_3'].unique()) + list(df['genre_4'].unique()) + list(df['genre_5'].unique())
genre_set = set(genre_list)

In [ ]:
#@title
genres_count = pd.DataFrame.from_dict({
    'count': [len(genre_set_pl), len(genre_set), 5071],
    'label': ['Spotify Polska', 'Spotify Global', 'Spotify ogółem']
})

# Prezentacja

## Metodologia

Ranking Spotify Charts jest tworzony automatycznie co tydzień na podstawie ilości odtworzeń utworu w poprzednim tygodniu.

## Sumaryczna liczba gatunków

In [ ]:
px.bar(genres_count, x='label', y='count') # https://everynoise.com/engenremap.html

## Rozkład gatunków 2016-2022

### Spotify Global

In [ ]:
sum1 = df['genre_1'].value_counts().reset_index()
sum2 = df['genre_2'].value_counts().reset_index()

In [ ]:
sum1= sum1.merge(sum2, on='index', how='left')

sum1['sum'] = sum1['genre_1'] + sum1['genre_2']

In [ ]:
px.pie(sum1[sum1['sum'] > 500], values='sum', names='index')

### Spotify Polska

In [ ]:
sum1 = df_pl['genre_1'].value_counts().reset_index()
sum2 = df_pl['genre_2'].value_counts().reset_index()

In [ ]:
sum1= sum1.merge(sum2, on='index', how='left')

sum1['sum'] = sum1['genre_1'] + sum1['genre_2']

In [ ]:
px.pie(sum1[sum1['sum'] > 500], values='sum', names='index')

## Rozkład top 10 gatunków w czasie

### Spotify Global

In [ ]:
import funcs

df = funcs.df_week_timestamp_to_monthly_period(df)

In [ ]:
data = df.groupby('week_timestamp')['genre_1'].apply(lambda x: x.value_counts().head(10)).reset_index(name='Count')
labels = data.week_timestamp.unique()

labels = map(lambda x: x.strftime('%Y-%m'), labels)
labels = list(labels)

In [ ]:
funcs.generate_pie_chart(data, labels)

<IPython.core.display.Javascript object>

### Spotify Polska

In [ ]:
df_pl = funcs.df_week_timestamp_to_monthly_period(df_pl)

In [ ]:
data_pl = df_pl.groupby('week_timestamp')['genre_1'].apply(lambda x: x.value_counts().head(10)).reset_index(name='Count')
labels_pl = data_pl.week_timestamp.unique()

labels_pl = map(lambda x: x.strftime('%Y-%m'), labels_pl)
labels_pl = list(labels_pl)

funcs.generate_pie_chart(data_pl, labels_pl)

<IPython.core.display.Javascript object>

## Liczba unikalnych gatunków w czasie

In [ ]:
genres_df = funcs.genres_to_one_column(df)
genres_df_pl = funcs.genres_to_one_column(df_pl)

data = genres_df.groupby('week_timestamp').nunique()
data_pl = genres_df_pl.groupby('week_timestamp').nunique()

data_pl['label'] = 'Spotify Polska'
data['label'] = 'Spotify Global'

data.index = data.index.to_timestamp().strftime("%Y-%m")
data_pl.index = data_pl.index.to_timestamp().strftime("%Y-%m")

data_final = pd.concat([data, data_pl], ignore_index=False)

funcs.generate_timeline_chart(data_final)

### Sezonowość

Na wykresie widać pewien trend, a mianowicie: w grudniu zawsze następuje skok w liczbie unikalnych gatunków muzyki.
Jest to spowodowane prawdopodobnie kolędami i świątecznymi piosenkami, które wykraczają poza obecną modę w muzyce popularnej.

### Przyczyna różnicy

Są dwie prawdopodobne przyczyny różnicy:

- techniczna, tj. modele sztucznej inteligencji lepiej klasyfikują gatunki dla zagranicznych piosenek
- globalna, tj. intuicyjną konsekwencją rozważania globalnego rankingu jest to, że będzie on bardziej zróżnicowanych niż ranking lokalny

## Najpopularniejsi artyści

Jako ilość unikalnych piosenek w danym miesiącu w rankingu.

In [ ]:
sub_pl = df_pl.drop_duplicates(subset=['track_uri', 'week_timestamp'])
sub = df.drop_duplicates(subset=['track_uri', 'week_timestamp'])

data_pl = sub_pl['artist_1_name'].value_counts().reset_index()
data = sub['artist_1_name'].value_counts().reset_index()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, subplot_titles=["Spotify Polska", "Spotify Global"])

fig.add_trace(
    go.Bar(x=data_pl['index'], y=data_pl['artist_1_name']),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=data['index'], y=data['artist_1_name'], ),
    row=1, col=2
)

fig['layout']['xaxis']['title']='Artysta'
fig['layout']['xaxis2']['title']='Artysta'
fig['layout']['yaxis']['title']='Ilość wystąpień w rankingu'
fig['layout']['yaxis2']['title']='Ilość wystąpień w rankingu'

fig.update_layout(height=600, width=1500, title_text="Najpopularniejsi artyści")
fig.show()

## Średni czas od daty premiery, a do pierwszego pojawienia się w Weekly Top 100

In [ ]:
df = funcs.calculate_days_diff(df)
df_pl = funcs.calculate_days_diff(df_pl)

df = df.drop_duplicates(subset=['track_uri'])
df_pl = df_pl.drop_duplicates(subset=['track_uri'])

df = df[df['date_diff'] > 0]
df_pl = df_pl[df_pl['date_diff'] > 0]

In [ ]:
#df = df[df['week_timestamp'].dt.month != 12]
#df_pl = df_pl[df_pl['week_timestamp'].dt.month != 12]

df['label'] = 'Spotify Global'
df_pl['label'] = 'Spotify Polska'

### Średnia

Średnia w przeciwieństwie do mediany jest podatna na anomalie/outliery. 

In [ ]:
data = pd.concat([df_pl, df], ignore_index=False)
data = funcs.df_period_to_string(data)

data_plot = data.groupby(['week_timestamp', 'label']).mean()

px.line(data_plot.reset_index(), x='week_timestamp', y='date_diff', color='label')

### Mediana

In [ ]:
data_plot = data.groupby(['week_timestamp', 'label']).median()

px.line(data_plot.reset_index(), x='week_timestamp', y='date_diff', color='label')

### Charakterystyka

In [ ]:
data.groupby(['label'])['date_diff'].describe()

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Spotify Global,4697.0,512.451778,2413.497847,1.0,6.0,6.0,24.0,27391.0
Spotify Polska,4336.0,546.886301,2399.676634,1.0,6.0,7.0,35.0,27391.0


### Wpływ czynników zewnętrznych (Tiktok)

Można znaleźć przypadki, gdy utwór trafia do notowań Top 100 nawet kiedy jego premiera odbyła się rok lub więcej temu. Warto zaznaczyć wpływ czynników zewnętrznych na ranking.

Wybierając z danych ostatnią zescrapowaną piosenkę, widzimy:

In [ ]:
df.iloc[-1]

week_timestamp                               2022-05
track_name              Only Love Can Hurt Like This
track_uri                     62ke5zFUJN6RvtXZgVH0F8
artist_1_name                           Paloma Faith
artist_2_name                                    NaN
artist_3_name                                    NaN
artist_1_uri                  4fwuXg6XQHfdlOdmw36OHa
artist_2_uri                                     NaN
artist_3_uri                                     NaN
total_streams                                5682774
genre_1                                 british soul
genre_2                                    dance pop
genre_3                                      europop
genre_4                                     neo soul
genre_5                                          NaN
this_position_streak                               1
total_streak                                       1
release_date                     2014-03-10 00:00:00
peak_date                        2022-05-26 00

Piosenka miała premierę 8 lat temu (!), ale w Top 100 na Spotify pojawiła się dopiero 26 maja 2022 roku. Możemy się zastanowić czemu tak się stało.

#### Google Trends

Analizując trendy wyszukiwań Google'a w poszukiwaniu przyczyny renesansu utworu "Only Love Can Hurt Like This", widać wyraźny skok zainteresowania w roku 2022. Pierwotne zainteresowanie piosenką trwało dwa lata (od premiery w marcu 2014 do marca 2016). W lutym 2022 piosenka nagle znowu się staje popularna.

In [ ]:
trends = pd.read_csv('trends.csv')

px.line(trends, x='Data', y='Zainteresowanie', title='Zainteresowanie w czasie frazą "Only Love Can Hurt Like This"')

Analizując przy okazji komentarze na serwisie Youtube, możemy zauważyć wiele komentarzy tego typu:

![e](https://i.imgur.com/8BTTnTK.png)

Komentarz sprzed 5 miesięcy temu pokrywa się ze skokiem zainteresowania w Google Trends. Zatem możemy sformułować hipotezę, że serwisy jak Tiktok nie pozostają bez znaczenia w kształtowaniu obecnych rankigów przebojów i przyczyniaja się do globalizacji mniej popularnych artystów. W obecnych czasach popularność może przyjść nawet do nieznanego nikomu artysty przy odrobinie szczęścia.

## Niektóre wnioski


* 50% (lub mniej) piosenek, które stają się hitami zazwyczaj trafiają do rankingu Spotify w ciągu tygodnia od premiery, a 75% (lub mniej) w ciągu miesiąca od premiery
    * (poniekąd) okres do miesiąca po premierze jest najważniejszy dla piosenki czy stanie się hitem
* Serwisy typu Tiktok przyczyniają się do popularności piosenek w rankingach
* W okresie świątecznym cały świat pogrąża się w świątecznym klimacie dzięki kolędom
* W Polsce na Spotify obecnie topowymi gatunkami jest polski rap/hip-hop. Dopiero dalej jest pop i rock.
    * przed 2018 dominował pop
* Na świecie najbardziej popularny pop, rap i muzyka latino.

# Podsumowanie

### Strona techniczna

Do analizy zostały wykorzystane trzy źródła danych:
1. Billboard Top 100 - amerykański ranking przebojów
2. Lista przebojów Trójki - polski ranking, w którym na utwory głosują słuchacze radia (voluntary response bias)
3. Spotify Charts Weekly - cotygodniowe podsumowanie najpopularniejszych utworów w serwisie streamingowym Spotify ( https://charts.spotify.com/charts/view/regional-global-weekly/2016-12-29 )

Zbiór danych dla (1) zawiera 500k elementów. (3) dla Polski i Świata 120k elementów.

Dane zostały pobrane wskutek scrapowania HTMLa (1) albo z publicznie dostępnego API (2, 3).



### Problemy

### LP3 jest przykładem stronniczości próby złożonej z ochotników

Próbka respodentów głosujących na Listę przebojów Trójki nieodwzwierciedla ogółu populacji. Co widać w porównaniu LP3 vs Spotify (LP3 ma większy udział rocka i ogólnie uznawanych "klasyków"). Ranking LP3 odwzwierciedla jedynie preferencje słuchaczy Trójki.

### Spotify jest obiektywniejszym rankingiem, ale nadal nie jest statystycznie reprezentacyjna

Różnica w gustach muzycznych, które widać w porównaniu LP3 vs Spotify wynika prawdopodobnie z różnicy wieku użytkowników serwisu.

#### Brak łatwego dostępu na określenie gatunku przynależącego do danego utworu

W internecie nie ma usług tego typu. Nawet jeżeli by były - napotkamy problem różnych identyfikatorów w dwóch zbiorach danych. Dopasowanie po nazwie zawsze będzie podatne na błędy.

#### Brak ustandaryzowanych gatunków i problem przetwarzania ich

Jedynym łatwym sposobem na określenie było Spotify. Spotify zaś zwraca listę przypisanych gatunków dla danego artysty. Pojawia się tu kilka problemów:
1. Zwracana jest lista gatunków
    * w jaki sposób to przechowywać w kolumnie? - przechowywanie listy jest złą praktyką
    * ilu elementów w liście się spodziewać? - jeżeli przyjmujemy, że rozdzielimy listę na kolumny to musimy się ograniczyć do **n** elementów i liczyć się z utratą informacji
2. Zwracane gatunki nie są "ustandaryzowane"
    * czy dream pop to podgatunek popu? czym się różni southern rap od missouri rap?
    * jeżeli elementy w zwracanej liście są posortowane losowo, w jaki sposób przyjąć "główny gatunek" dla danego utworu bez ręcznego mapowania?
    * by łatwo zwizualizować powyższe zależności wymagana jest hierarchiczna struktura danych. W przypadku tej analizy jest to zadanie wymagające ręcznej ingerencji, a zatem zbyt czasochłonne
4. Występy gościnne
    * jeżeli w utworze występuje 2 czy 3 artystów to mogą być oni gatunkowo różni - którego artystę przyjmiemy za punkt odniesienia dla gatunków?
5. Rate limiting API
    * mając 500k elementów pojawia się problem z nałożonym limitem zapytań dla pojedynczego użytkownika 